In [ ]:
!pip install -U google-adk

In [ ]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from typing import List, Dict

In [ ]:
from google.cloud import firestore

In [ ]:
from google.adk.runners import InMemoryRunner
print([m for m in dir(InMemoryRunner) if "send" in m])

[]


In [ ]:
%%writefile db.py
import firebase_admin
from firebase_admin import credentials, firestore
from typing import Optional, Dict

class UserProfileDB:
    def __init__(self, service_account_path: str):
        """
        Initialize Firestore using Firebase Admin SDK.
        You must pass in the path to your service account JSON.
        """
        if not firebase_admin._apps:
            cred = credentials.Certificate(service_account_path)
            firebase_admin.initialize_app(cred)

        self.db = firestore.client()
        self.collection = self.db.collection("user_profiles")

    def get_profile(self, user_id: str) -> Optional[Dict]:
        """
        Fetch the user's profile. Returns None if the document does not exist.
        """
        doc = self.collection.document(user_id).get()
        return doc.to_dict() if doc.exists else None

    def update_field(self, user_id: str, field_path: str, value):
        """
        Update a specific field in the user's profile.
        Supports nested fields using dot notation.
        Example: update_field("u1", "skin.type", "oily")
        """
        self.collection.document(user_id).set(
            {field_path: value},
            merge=True
        )

    def save_profile(self, user_id: str, profile: dict):
        """
        Overwrite or merge a complete user profile dictionary.
        """
        self.collection.document(user_id).set(profile, merge=True)

Overwriting db.py


In [ ]:
!ls -l

total 16
-rw-r--r-- 1 root root 1421 Nov 24 13:14 db.py
drwxr-xr-x 2 root root 4096 Nov 24 12:53 __pycache__
drwxr-xr-x 1 root root 4096 Nov 20 14:30 sample_data
-rw-r--r-- 1 root root 2376 Nov 24 12:46 skinbuddy-ai-firebase-adminsdk-fbsvc-b86d3ad042.json


In [ ]:
from db import UserProfileDB

In [ ]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini

In [ ]:
def update_field(context: dict, field_name: str, value):
    user_id = context["user_id"]   # ADK injects this from the session
    db.update_field(user_id, field_name, value)
    return {"status": "success", "user_id": user_id}

In [ ]:
intake_agent = Agent(
    name="skincare_intake_agent",
    instruction="""
    You are a skincare intake agent.
    Ask the user step-by-step questions to build their skincare profile.

    Steps:
    1. Ask for skin type.
    2. Validate skin type (oily, dry, normal, combination).
    3. Ask for skin concerns.
    4. Validate concerns.
    5. Store results using update_field tool.
    """,
    model=Gemini(model="gemini-2.5-flash"),
    tools=[update_field]
)


In [ ]:
def start_intent(context):
    return {
        "response": "Hi! Let's build your skincare profile. What is your skin type? (oily, dry, normal, combination)"
    }

In [ ]:
runner = InMemoryRunner(agent=intake_agent, app_name="agents")

    # Create session
session = await runner.session_service.create_session(
        app_name="agents",
        user_id="test_user1"
    )


In [ ]:
#from google.adk.runners import InMemoryRunner

#runner = InMemoryRunner(
#    agent=intake_agent
#)

response = await runner.run_debug("start")


 ### Created new session: debug_session_id

User > start


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Invalid JSON payload received. Unknown name "additional_properties" at \'tools[0].function_declarations[0].parameters.properties[0].value\': Cannot find field.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'tools[0].function_declarations[0].parameters.properties[0].value', 'description': 'Invalid JSON payload received. Unknown name "additional_properties" at \'tools[0].function_declarations[0].parameters.properties[0].value\': Cannot find field.'}]}]}}

In [ ]:
response = await runner.run_debug({
    "normal"
})


 ### Continue session: debug_session_id

User > normal


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Invalid JSON payload received. Unknown name "additional_properties" at \'tools[0].function_declarations[0].parameters.properties[0].value\': Cannot find field.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'tools[0].function_declarations[0].parameters.properties[0].value', 'description': 'Invalid JSON payload received. Unknown name "additional_properties" at \'tools[0].function_declarations[0].parameters.properties[0].value\': Cannot find field.'}]}]}}

In [ ]:
response = await runner.run_debug({
    "Aging"
})


 ### Continue session: debug_session_id

User > Aging
skincare_intake_agent > It looks like I'm missing your user ID to save your preferences. Could you please provide your user ID so I can record your skin type as "normal" and your primary concern as "anti-aging"?


#test

In [ ]:
from db import UserProfileDB

db = UserProfileDB("/content/skinbuddy-ai-firebase-adminsdk-fbsvc-b86d3ad042.json")

In [ ]:
db.save_profile("test_user", {"skin_type": "oily"})
db.get_profile("test_user")

{'skin_type': 'oily'}

In [ ]:
profile = db.get_profile("test_user")
print(profile)

{'skin_type': 'oily'}
